# GenAI Development Environment - Quick Start

This notebook demonstrates the key features of your GenAI development environment.

## 1. Test Local LLM with Ollama

In [ ]:
import ollama
import requests

# Test Ollama connection
try:
    response = requests.get('http://ollama:11434/api/tags')
    if response.status_code == 200:
        models = response.json()
        print("✅ Ollama is running!")
        print(f"📦 Available models: {[model['name'] for model in models.get('models', [])]}")
    else:
        print("❌ Ollama is not responding")
except Exception as e:
    print(f"❌ Error connecting to Ollama: {e}")
    print("💡 Make sure to run './setup-models.sh' to download models")

In [ ]:
# Chat with local model (requires model to be downloaded)
try:
    response = ollama.chat(
        model='qwen2.5:7b',  # Change to available model
        messages=[
            {'role': 'user', 'content': 'Explain machine learning in simple terms'}
        ]
    )
    print("🤖 Local LLM Response:")
    print(response['message']['content'])
except Exception as e:
    print(f"❌ Error: {e}")
    print("💡 Download models first: './setup-models.sh'")

## 2. Test Vector Database (Qdrant)

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from sentence_transformers import SentenceTransformer
import numpy as np

# Connect to Qdrant
try:
    client = QdrantClient(host="qdrant", port=6333)
    print("✅ Connected to Qdrant")
    
    # Get collections
    collections = client.get_collections()
    print(f"📂 Collections: {collections}")
    
except Exception as e:
    print(f"❌ Error connecting to Qdrant: {e}")

In [ ]:
# Create embeddings and store in Qdrant
try:
    # Load embedding model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    # Sample documents
    documents = [
        "Machine learning is a subset of artificial intelligence",
        "Neural networks are inspired by biological neurons",
        "Deep learning uses multiple layers of neural networks",
        "Natural language processing helps computers understand text"
    ]
    
    # Generate embeddings
    embeddings = model.encode(documents)
    print(f"✅ Generated {len(embeddings)} embeddings of dimension {embeddings.shape[1]}")
    
    # Create collection
    collection_name = "test_collection"
    
    try:
        client.delete_collection(collection_name)
    except:
        pass
        
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=embeddings.shape[1], distance=Distance.COSINE)
    )
    
    # Insert vectors
    client.upsert(
        collection_name=collection_name,
        points=[
            {
                "id": i,
                "vector": embedding.tolist(),
                "payload": {"text": doc}
            }
            for i, (embedding, doc) in enumerate(zip(embeddings, documents))
        ]
    )
    
    print(f"✅ Stored {len(documents)} documents in Qdrant")
    
except Exception as e:
    print(f"❌ Error: {e}")

In [ ]:
# Search similar documents
try:
    query = "What is artificial intelligence?"
    query_embedding = model.encode([query])[0]
    
    results = client.search(
        collection_name=collection_name,
        query_vector=query_embedding.tolist(),
        limit=3
    )
    
    print(f"🔍 Search results for: '{query}'")
    for result in results:
        print(f"  📄 Score: {result.score:.3f} - {result.payload['text']}")
        
except Exception as e:
    print(f"❌ Error: {e}")

## 3. Test LangChain with Local Models

In [ ]:
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Initialize local LLM
try:
    llm = OllamaLLM(
        model="qwen2.5:7b",  # Change to your available model
        base_url="http://ollama:11434"
    )
    
    # Test simple prompt
    response = llm.invoke("What are the benefits of using local LLMs?")
    print("🤖 LangChain + Local LLM Response:")
    print(response)
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("💡 Make sure you have downloaded a model with './setup-models.sh'")

## 4. Test Database Connections

In [ ]:
import psycopg2
import redis
from elasticsearch import Elasticsearch

# Test PostgreSQL
try:
    conn = psycopg2.connect(
        host="postgres",
        database="genai_db", 
        user="genai_user",
        password="genai_password"
    )
    cursor = conn.cursor()
    cursor.execute("SELECT version();")
    version = cursor.fetchone()
    print(f"✅ PostgreSQL: {version[0][:50]}...")
    conn.close()
except Exception as e:
    print(f"❌ PostgreSQL Error: {e}")

# Test Redis
try:
    r = redis.Redis(host='redis', port=6379, db=0)
    r.set('test_key', 'GenAI Dev Environment')
    value = r.get('test_key').decode('utf-8')
    print(f"✅ Redis: {value}")
except Exception as e:
    print(f"❌ Redis Error: {e}")

# Test Elasticsearch
try:
    es = Elasticsearch([{'host': 'elasticsearch', 'port': 9200}])
    info = es.info()
    print(f"✅ Elasticsearch: {info['version']['number']}")
except Exception as e:
    print(f"❌ Elasticsearch Error: {e}")

In [ ]:
# Test Qdrant Vector Database
try:
    client = QdrantClient(host="qdrant", port=6333)
    print("✅ Qdrant: Connected successfully")
    collections = client.get_collections()
    print(f"📂 Collections: {len(collections.collections)} found")
except Exception as e:
    print(f"❌ Qdrant Error: {e}")

# Test Ollama Local LLM Server
try:
    response = requests.get('http://ollama:11434/api/version')
    if response.status_code == 200:
        version_info = response.json()
        print(f"✅ Ollama: Version {version_info.get('version', 'unknown')}")
        
        # List available models
        models_response = requests.get('http://ollama:11434/api/tags')
        if models_response.status_code == 200:
            models = models_response.json()
            model_list = [model['name'] for model in models.get('models', [])]
            print(f"🤖 Available models: {model_list if model_list else 'No models downloaded yet'}")
        else:
            print("📦 No models loaded yet")
    else:
        print("❌ Ollama is not responding")
except Exception as e:
    print(f"❌ Ollama Error: {e}")
    print("💡 Make sure to run './setup-models.sh' after environment starts")

## 5. Test OpenAI API (if configured)

In [ ]:
import os
from openai import OpenAI

# Test OpenAI API (requires API key in .env)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key != 'your_openai_api_key_here':
    try:
        client = OpenAI(api_key=api_key)
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": "Say hello from the GenAI dev environment!"}
            ],
            max_tokens=50
        )
        print("✅ OpenAI API Response:")
        print(response.choices[0].message.content)
    except Exception as e:
        print(f"❌ OpenAI API Error: {e}")
else:
    print("💡 OpenAI API key not configured. Add it to .env file to test.")

## 🎉 Environment Test Complete!

If all tests above passed, your GenAI development environment is ready!

### Next Steps:
1. Download AI models: `./setup-models.sh`
2. Configure API keys in `.env` file
3. Explore the example projects in `/workspace/projects/`
4. Build your own AI applications!

### Useful Resources:
- 📊 Qdrant Dashboard: http://localhost:6333/dashboard
- 🔍 Elasticsearch: http://localhost:9200
- 🤖 Ollama API: http://localhost:11434
- 💻 VS Code: http://localhost:8080